In [2]:
# Import packages
import pandas as pd
import numpy as np
import requests

In [3]:
# Set dictionaries for organizing data
statedict = {'AL': '1', 'AK': '2', 'AZ': '4', 'AR': '5', 'CA': '6', 'CO': '8', 'CT': '9', 'DE': '10', 'DC': '11', 'FL': '12',
             'GA': '13', 'HI': '15', 'ID': '16', 'IL': '17', 'IN': '18', 'IA': '19', 'KS': '20', 'KY': '21', 'LA': '22', 
             'ME':'23', 'MD': '24', 'MA': '25', 'MI': '26', 'MN': '27', 'MS': '28', 'MO': '29', 'MT': '30', 'NE': '31', 'NV': 
             '32', 'NH': '33', 'NJ': '34', 'NM': '35', 'NY': '36', 'NC': '37', 'ND': '38', 'OH': '39', 'OK': '40', 'OR': '41', 
             'PA': '42', 'RI': '44', 'SC': '45', 'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49', 'VT': '50', 'VA': '51', 'WA': 
             '53', 'WV': '54', 'WI': '55', 'WY': '56'}
rvsestatedict = {v: k for k, v in statedict.items()}

In [4]:
# Read in data from Census API
def get_data(yr):
    url = (f"http://api.census.gov/data/{yr}/cps/voting/nov")
    # Get parameters - some are called different variables in different years
    if (yr == '1994'):
        param_list = "PES3,PES4,GEMETSTA,GEREG,PERACE,PRHSPNON,PESEX,PRTAGE,PEMARITL,HRNUMHOU,PEAFNOW,PEEDUCA,HUFAMINC,PREXPLF,PRFTLF"
    elif(yr == '1996' or yr == '1998' or yr == '2000' or yr == '2002'):
        param_list = "PES1,PES2,GEMETSTA,GEREG,PERACE,PRHSPNON,PESEX,PRTAGE,PEMARITL,HRNUMHOU,PEAFNOW,PEEDUCA,HUFAMINC,PREXPLF,PRFTLF"
    elif(yr == '2004' or yr == '2006' or yr == '2008'):
        param_list = "PES1,PES2,GTMETSTA,GEREG,PTDTRACE,PEHSPNON,PESEX,PRTAGE,PEMARITL,HRNUMHOU,PEAFNOW,PEEDUCA,HUFAMINC,PREXPLF,PRFTLF"
    else:
        param_list = "PES1,PES2,GTMETSTA,GEREG,PTDTRACE,PEHSPNON,PESEX,PRTAGE,PEMARITL,HRNUMHOU,PEAFNOW,PEEDUCA,HEFAMINC,PREXPLF,PRFTLF"

    r = requests.get(url,
                     params = {"get": param_list,
                               "for": "state:*"})
    
    census_df = pd.DataFrame(data = r.json())
    census_df.rename(columns = census_df.iloc[0], inplace = True)
    census_df.drop([0], axis = 0, inplace = True)
    # Change column names
    census_df.columns = ["Voted", "Registered_to_Vote", "Metropolitan",
                         "Geographic_Region", "Race", "Hispanic",
                         "Female", "Age", "Marital_Status", 
                         "Household_Members", "In_Armed_Forces", 
                         "Education_Completed", "Family_Income_category", "Employment_Status", 
                         "Full_Time", "State"]
    # Replace number with state abbreviation
    census_df.replace({'State': rvsestatedict}, inplace = True)
    # Change column types
    census_df = census_df.astype({"Voted": int, "Registered_to_Vote": int, "Metropolitan": int, 
                                  "Geographic_Region": int, "Race": int, "Hispanic": int, "Female": int,
                                  "Age" : int, "Marital_Status": int, "Household_Members": int, 
                                  "In_Armed_Forces": int, "Education_Completed": int,
                                  "Family_Income_category": int, "Employment_Status": int, "Full_Time": int, "State": str})
    col2 = census_df.pop('State')
    census_df.insert(0, 'State', col2)
    
    census_df.Voted.replace({2: 0, -1: np.nan, -3: np.nan, -9: np.nan, -2: np.nan},inplace = True)
    census_df.Registered_to_Vote.replace({2: 0, -1: np.nan, -3: np.nan, -9: np.nan, -2: np.nan}, inplace = True)
    # All people who voted were registered to vote, so need to change any NA's in Registered_to_Vote to 1 where Voted is 1
    census_df.loc[census_df.Voted == 1, 'Registered_to_Vote'] = 1
    census_df.Female.replace({2: 0},inplace = True)
    census_df.Hispanic.replace({2: 0}, inplace = True)
    census_df.Race = np.where(census_df.Race > 4, "Other", census_df.Race)
    census_df.Race.replace({'1': "White", '2': "Black", '4': "Asian", '3': "American Indian, Aluet, Eskimo"}, inplace = True)
    census_df.Marital_Status.replace({2: 1, 3: 0, 4: 0, 5: 0, 6: 0, -1: np.nan},inplace = True)
    census_df.In_Armed_Forces.replace({2: 0, -1: np.nan},inplace = True)
    census_df.Employment_Status.replace({2: 0, -1: np.nan, 127: np.nan},inplace = True)
    census_df.Full_Time.replace({2: 0, -1: np.nan, 127: np.nan},inplace = True)
    census_df.Metropolitan.replace({2: 0, 3: np.nan}, inplace = True)
    census_df.Geographic_Region.replace({1: "Northeast", 2: "Midwest", 3: "South", 4: "West"}, inplace = True)
    census_df.Education_Completed = np.where(census_df.Education_Completed < 39, "No_HS_Diploma", census_df.Education_Completed)
    census_df.Education_Completed.replace({'46': "Doctorate", '44': "Masters",
                                           '39': "HighSchool/GED",'42': "Associates", 
                                           '40': "Some_College",'-1': np.nan, '43': "Bachelors",
                                           '45': "Professional_School",'41': "Associates"},inplace = True)
    census_df.Family_Income_category.replace({-1: np.nan, -2: np.nan, -3: np.nan}, inplace = True)

    # Create different version of family income variable, 1 if greater than $50,000, 0 if less
    census_df['Family_Income_dummy'] = np.where(census_df.Family_Income_category > 11, 1, 0)
    # Create different version of family income variable that is the middle of the range (or highest of the range in the case of the lowest value, lowest of the range in the case of the highest value)
    census_df['Family_Income_actual'] = census_df.Family_Income_category.replace({1: 5000, 2: 6250, 3: 8250, 4: 11250, 5: 13750, 
                                                                               6: 17500, 7: 22500, 8: 27500, 9: 32500, 10: 37500,
                                                                               11: 45000, 12: 55000, 13: 67500, 14: 87500, 
                                                                               15: 125000, 16: 150000})
    census_df['Year'] = yr
    
    return census_df

In [5]:
full_table = get_data("1994")

In [6]:
years = ['1996', '1998', '2000', '2002', '2004', '2006', '2008', '2010', '2012', '2014', '2016', '2018', '2020']
for year in years:
    new_tab = get_data(year)
    full_table = pd.concat([full_table, new_tab])

In [8]:
full_table.head()

,State,Voted,Registered_to_Vote,Metropolitan,Geographic_Region,Race,Hispanic,Female,Age,Marital_Status,Household_Members,In_Armed_Forces,Education_Completed,Family_Income_category,Employment_Status,Full_Time,Family_Income_dummy,Family_Income_actual,Year
1,AL,0.0,0.0,1.0,South,Black,0,0,83,0.0,3,0.0,No_HS_Diploma,2.0,NaN,NaN,0,6250.0,1994
2,AL,0.0,1.0,1.0,South,Black,0,0,25,0.0,3,0.0,HighSchool/GED,2.0,1.0,0.0,0,6250.0,1994
3,AL,NaN,NaN,1.0,South,Black,0,1,8,NaN,3,NaN,No_HS_Diploma,2.0,NaN,NaN,0,6250.0,1994
4,AL,0.0,1.0,1.0,South,Black,0,1,48,1.0,6,0.0,No_HS_Diploma,4.0,0.0,1.0,0,11250.0,1994
5,AL,0.0,0.0,1.0,South,Black,0,0,45,1.0,6,0.0,HighSchool/GED,4.0,1.0,1.0,0,11250.0,1994


In [9]:
full_table_aggregates = full_table.groupby(['State', "Year", 'Geographic_Region', "Race", "Female", "Marital_Status", 
                                            "In_Armed_Forces", "Education_Completed", "Employment_Status",
                                            "Family_Income_actual"])['Voted'].agg(['sum','count'])

In [13]:
full_table_aggregates

sum  \
State Year Geographic_Region Race                           Female Marital_Status In_Armed_Forces Education_Completed Employment_Status Family_Income_actual         
AK    1994 West              American Indian, Aluet, Eskimo 0      0.0            0.0             Associates          1.0               27500.0                0.0   
                                                                                                                                        55000.0                1.0   
                                                                                                  Bachelors           1.0               45000.0                0.0   
                                                                                                                                        67500.0                1.0   
                                                                                                  HighSchool/GED      0.0               11250.0                0.0   
...                                                                                                                                                            ...   
WY    2020 West              White                          1      1.0            0.0             Some_College        1.0               55000.0                3.0   
                                                                                                                                        67500.0                6.0   
                                                                                                                                        87500.0                6.0   
                                                                                                                                        125000.0              12.0   
                                                                                                                                        150000.0               3.0   

                                                                                                                                                              count  
State Year Geographic_Region Race                           Female Marital_Status In_Armed_Forces Education_Completed Employment_Status Family_Income_actual         
AK    1994 West              American Indian, Aluet, Eskimo 0      0.0            0.0             Associates          1.0               27500.0                   1  
                                                                                                                                        55000.0                   1  
                                                                                                  Bachelors           1.0               45000.0                   1  
                                                                                                                                        67500.0                   1  
                                                                                                  HighSchool/GED      0.0               11250.0                   1  
...                                                                                                                                                             ...  
WY    2020 West              White                          1      1.0            0.0             Some_College        1.0               55000.0                   3  
                                                                                                                                        67500.0                   7  
                                                                                                                                        87500.0                   9  
                                                                                                                                        125000.0                 13  
        

In [17]:
full_table_aggregates.shape

(285741, 2)

In [34]:
full_table.to_csv("census_voters.csv", index = False)

In [18]:
full_table_aggregates.to_csv("census_voters_aggregates.csv")